In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit

In [2]:
def generate_substractor(n):
    qrX = QuantumRegister(n, name='g')
    qrY = QuantumRegister(n, name='a') # output of substraction should be put to register |a>
    # carry is unnecessary here
    
    qc = QuantumCircuit(qrY, qrX)
    
    # 3.6
    for i in range(n-1, -1, -1):
        qc.cx(qrX[i], qrY[i])

    # 3.5
    for i in range(n-1, 1, -1):
        qc.cx(qrX[i-1], qrX[i])

    # 3.4
    for i in range(1, n):
        qc.ccx(qrX[i-1], qrY[i-1], qrX[i])
        qc.cx(qrX[i], qrY[i])

    # 3.3
    for i in range(n-1, 0, -1):
        qc.ccx(qrX[i-1], qrY[i-1], qrX[i])

    # 3.2
    for i in range(1, n-1):
        qc.cx(qrX[i], qrX[i+1])

    # 3.1
    for i in range(n-1, 0, -1):
        qc.cx(qrX[i], qrY[i])
        
    return qc

In [3]:
# qc = generate_substractor(4)
# qc.draw(output='mpl')

In [4]:
def generate_negation_part(n):
    qrG = QuantumRegister(n, name='g')
    
    qc = QuantumCircuit(qrG)
    
    qc.barrier(qrG[:])
    for i in range(n):
        qc.x(qrG[i])
    qc.barrier(qrG[:])
    
    return qc

In [5]:
# qc = generate_negation_part(4)
# qc.draw(output='mpl')

In [6]:
def generate_incrementator(n):
    qc_sub = generate_substractor(n)
    qc_neg = generate_negation_part(n)
    
    return qc_sub + qc_neg + qc_sub + qc_neg

In [7]:
# qc = generate_incrementator(4)
# qc.draw(output='mpl')

In [8]:
def get_bits(x, n):
    return [int(x) for x in '{:0{size}b}'.format(x, size=n)]

In [9]:
def generate_init_part(a, g, n):
    a_bits = get_bits(a, n)
    g_bits = get_bits(g, n)
    
    qrA = QuantumRegister(n, name='a')
    qrG = QuantumRegister(n, name='g')
    qc = QuantumCircuit(qrA, qrG)
    
    for i in range(n):
        if a_bits[n-(i+1)] == 1:
            qc.x(qrA[i])

    for i in range(n):
        if g_bits[n-(i+1)] == 1:
            qc.x(qrG[i])

    qc.barrier(qrA[:], qrG[:])
    
    return qc

In [10]:
def generate_measure_part(n):
    qrA = QuantumRegister(n, name='a')
    qrG = QuantumRegister(n, name='g')
    crA = ClassicalRegister(n, name='aValue')
    crG = ClassicalRegister(n, name='gValue')

    qc = QuantumCircuit(qrA, qrG, crA, crG)
    qc.barrier(qrA[:], qrG[:])
    qc.measure(qrA[:], crA[:])
    qc.measure(qrG[:], crG[:])
    
    return qc

In [11]:
# qc_meas = generate_measure_part(4)
# qc_meas.draw(output='mpl')

In [12]:
from qiskit import Aer
from qiskit import execute

backend = Aer.get_backend('qasm_simulator')

In [13]:
def test_incrementation(a, n, a_value):
    expected_value = (a + 1) % (2 ** n)
    print(expected_value, a_value, expected_value == a_value)

In [14]:
def test(a, g, n):
    init = generate_init_part(a, g, n)
    inc_gate = generate_incrementator(n)
    meas = generate_measure_part(n)

    qc = init + inc_gate + meas

    job = execute(qc, backend, shots=1)
    result = job.result()
    values = list(result.get_counts(qc).keys())

    assert len(values) == 1
    values = [int(v, 2) for v in values[0].split(' ')]

    g_value = values[0]
    a_value = values[1]

    test_incrementation(a, n, a_value)
    print(g, g_value, g == g_value)

In [15]:
test(6, 5, 3)

<ipython-input-6-3a680a89c908>:5: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return qc_sub + qc_neg + qc_sub + qc_neg
/home/bartek/MGR/masterThesisProject/venv/lib/python3.8/site-packages/qiskit/circuit/quantumcircuit.py:869: DeprecationWarning: The QuantumCircuit.combine() method is being deprecated. Use the compose() method which is more flexible w.r.t circuit register compatibility.
  return self.combine(rhs)


7 7 True
5 5 True


In [16]:
# for n in range(4):
#     n = n + 1
#     N = 2 ** n
#     for a in range(N):
#         for g in range(N):
#             print('---', a, g, '---')
#             test(a, g, n)